***

**<center><font size = "6">Build and Optimize Data Warehouses with BigQuery<center>**
***
<center><font size = "2">Prepared by: Sitsawek Sukorn<center>

### BigQuery: Qwik Start - Command Line

### Examine a table

In [ ]:
# Run in shell 
bq show bigquery-public-data:samples.shakespeare

+ bq to invoke the BigQuery command line tool
+ show is the action
+ Then you're listing the name of the project:public dataset.table in BigQuery that you want to see.

### Run the help command

In [ ]:
bq help query

### Run a query

Now you'll run a query to see how many times the substring "raisin" appears in Shakespeare's works.

+ To run a query, run the command bq query "[SQL_STATEMENT]":

+ Escape any quotation marks inside the [SQL_STATEMENT] with a \ mark, or

+ Use a different quotation mark type than the surrounding marks ("versus").

+ Run the following standard SQL query in Cloud Shell to count the number of times that the substring "raisin" appears in all of Shakespeare's works:

In [ ]:
bq query --use_legacy_sql=false \
'SELECT
   word,
   SUM(word_count) AS count
 FROM
   `bigquery-public-data`.samples.shakespeare
 WHERE
   word LIKE "%raisin%"
 GROUP BY
   word'

+ --use_legacy_sql=false makes standard SQL the default query syntax.

+ If you search for a word that isn't in Shakespeare's works, no results are returned.

+ Run the following search for "huzzah", returns no matches:



In [ ]:
bq query --use_legacy_sql=false \
'SELECT
   word
 FROM
   `bigquery-public-data`.samples.shakespeare
 WHERE
   word = "huzzah"'

### Create a new table

#### Create a new dataset
+ Use the bq ls command to list any existing datasets in your project:

In [ ]:
bq ls

+ Run bq ls and the bigquery-public-data Project ID to list the datasets in that specific project, followed by a colon (:):

In [ ]:
bq ls bigquery-public-data:

+ Use the bq mk command to create a new dataset named babynames in your Qwiklabs project:

In [ ]:
bq mk babynames

+ Run bq ls to confirm that the dataset now appears as part of your project:

In [ ]:
bq ls

#### Upload the dataset

+ Run this command to add the baby names zip file to your project, using the URL for the data file:

In [ ]:
curl -LO http://www.ssa.gov/OACT/babynames/names.zip

+ List the file:

In [ ]:
ls

+ Now unzip the file:

In [ ]:
unzip names.zip

+ That's a pretty big list of text files! List the files again:

In [ ]:
ls

+ Create your table:

In [ ]:
bq load babynames.names2010 yob2010.txt name:string,gender:string,count:integer

+ Run bq ls and babynames to confirm that the table now appears in your dataset:

In [ ]:
bq ls babynames

+ Run bq show and your dataset.table to see the schema:

In [ ]:
bq show babynames.names2010

+ Note: By default, when you load data, BigQuery expects UTF-8 encoded data. If you have data that is in ISO-8859-1 (or Latin-1) encoding and are having problems with your loaded data, you can tell BigQuery to treat your data as Latin-1 explicitly, using the -E flag. Learn more about Character Encodings from the Introduction to loading data guide.

### Run queries

+ Run the following command to return the top 5 most popular girls names:

In [ ]:
bq query "SELECT name,count FROM babynames.names2010 WHERE gender = 'F' ORDER BY count DESC LIMIT 5"

+ Run the following command to see the top 5 most unusual boys names:

In [ ]:
bq query "SELECT name,count FROM babynames.names2010 WHERE gender = 'M' ORDER BY count ASC LIMIT 5"

+ Note: The minimum count is 5 because the source data omits names with fewer than 5 occurrences.

### Clean up

+ Run the bq rm command to remove the babynames dataset with the -r flag to delete all tables in the dataset:

In [ ]:
bq rm -r babynames

+ Confirm the delete command by typing Y.

***

**<center><font size = "6">Creating a Data Warehouse Through Joins and Unions<center>**
***